In [2]:
import pandas as pd
from src.data_processing import detect_outliers

In [2]:
# Load the dataset (Parquet format)
df = pd.read_parquet('../charging_session_features.parquet')
# df = pd.read_parquet('../results/latest_result_markus.parquet')


In [3]:
# Feature Engineering
df['is_weekend'] = df['weekday'].apply(lambda x: 1 if x in [5, 6] else 0)  # Weekend feature

In [4]:
# User-based feature engineering
df['trip_count_per_weekday'] = df.groupby('c_vin')['weekday'].transform('count')
# df['avg_distance_per_dayofweek'] = df.groupby(['c_vin', 'weekday'])['distance'].transform('mean')

In [5]:
# Historical average features
df['mean_consumption'] = df.groupby('c_vin')['soc_demand'].transform('mean')
df['mean_duration'] = df.groupby('c_vin')['c_totalchargingdurationsec'].transform('mean')

In [7]:
# Save processed data as Parquet
df.to_parquet('../results/processed_data.parquet', index=False)

In [ ]:
# Remove outliers
num_features = ['c_realstartsoc', 'weekday_numerical', 'mean_consumption',
                'mean_duration', 'mean_dep_time', 'delta_soc_real',
                'plugin_duration_hr']
drop = detect_outliers(df, 0, ['delta_soc_real'])
df = df.drop(drop, axis=0).reset_index(drop=True)
print(f'After removing energy need(delta soc)outliers: {len(df)}')

drop = detect_outliers(df, 0, ['plugin_duration_hr'])
df = df.drop(drop, axis=0).reset_index(drop=True)
print(f'After removing duration outliers: {len(df)}')

df.to_parquet('results/removed_outliers.parquet')